In [ ]:
import ctypes
k32 = ctypes.WinDLL("kernel32.dll")
wlanapi = ctypes.WinDLL("Wlanapi.dll")

In [ ]:
import pywintypes
import ctypes.wintypes as wt

H = wt.HANDLE
DWORD = wt.DWORD
PVOID = ctypes.c_void_p
WCHAR = wt.WCHAR
WLAN_INTERFACE_STATE = ctypes.c_uint
BYTE = wt.BYTE
WORD = wt.WORD
HANDLE = wt.HANDLE



class GUID(ctypes.Structure):
    _fields_ = [
        ('Data1', DWORD),
        ('Data2', WORD),
        ('Data3', WORD),
        ('Data4', BYTE*8)
    ]

class WLAN_INTERFACE_INFO(ctypes.Structure):
    _fields_ = [
        ('InterfaceGuid', GUID),
        ('strInterfaceDescription', WCHAR),
        ('isState', WLAN_INTERFACE_STATE)
    ]

class PWLAN_INTERFACE_INFO_LIST(ctypes.Structure):
    _fields_ = [
        ("dwNumberOfItems", ctypes.POINTER(DWORD)),
        ('dwIndex', DWORD),
        ('InterfaceInfo', ctypes.POINTER(WLAN_INTERFACE_INFO))
    ]
    
PINFO = ctypes.POINTER(PWLAN_INTERFACE_INFO_LIST)
PDWORD = ctypes.POINTER(DWORD)
PHANDLE = ctypes.POINTER(HANDLE)

# class PWLAN_AVAILABLE_NETWORK_LIST(ctypes.Structure):
#     _fields_ = [
#         ('dwNumberofItems', DWORD),
#         ('dwIndex', DWORD),
#         ('Network', WLAN_AVAILABLE_NETWORK*20)
#     ]

In [ ]:
WlanEnumInterfaces = wlanapi.WlanEnumInterfaces
WlanEnumInterfaces.argtypes = [HANDLE, PVOID, PINFO]
WlanEnumInterfaces.restype = DWORD

WlanOpenHandle = wlanapi.WlanOpenHandle
WlanOpenHandle.argtypes = [DWORD, PVOID, PDWORD, PHANDLE]
WlanOpenHandle.restype = DWORD

# WlanGetAvailableNetworkList = wlanapi.WlanGetAvailableNetworkList
# WlanGetAvailableNetworkList.argtypes = [HANDLE, GUID, DWORD, PVOID, PWLAN_AVAILABLE_NETWORK_LIST ]
# WlanGetAvailableNetworkList.restype = DWORD

WlanScan = wlanapi.WlanScan
WlanScan.argtypes = [HANDLE, GUID, PVOID, PVOID, PVOID]
WlanScan.restype = DWORD

In [ ]:
dwClientVersion = 2
pres = None
#pdwNegotiatedVersion = PDWORD(ctypes.c_ulong(2))
#phClientHandle = PHANDLE(None)

neg_version = ctypes.c_ulong(0)
client_handle = HANDLE(0)
br = ctypes.byref
WlanOpenHandle(dwClientVersion, pres, br(neg_version), br(client_handle))

In [ ]:
ppi = PWLAN_INTERFACE_INFO_LIST()
WlanEnumInterfaces(client_handle, None, br(ppi))

In [ ]:
ppi.dwNumberOfItems.contents

In [ ]:
guid = ppi.InterfaceInfo

In [ ]:
WlanScan(client_handle, br(guid), None, None, None)

In [ ]:
ppi.dwIndex, ppi.dwNumberOfItems.contents
ppi.InterfaceInfo[0].InterfaceGuid.Data4

In [ ]:
import subprocess
def get_current_wifi2():
    lines = subprocess.getoutput("netsh wlan show interfaces").splitlines()
    ilines = iter(lines)
    i = 0
    def nl(ilines):
        line = next(ilines, None)
        if line is None:
            return None, None
        try:
            key, val = line.split(":", 1)
        except ValueError as e:
            return "", ""
        key = key.strip()
        val = val.strip()
        return key, val

    k = v = 1
    while k is not None:
        k, v = nl(ilines)
        if k == "Name" and v == "Wi-Fi":
            while True:
                k, v = nl(ilines)
                if k == "SSID":
                    return v
                
def get_current_wifi():
    lines = subprocess.getoutput("netsh wlan show interfaces").splitlines()
    name = None
    for line in lines:
        try:
            key, val = line.split(":", 1)
        except ValueError:
            continue
        key = key.strip()
        val = val.strip()
        if key == "Name":
            name = val
            continue
        if key == "SSID" and name == "Wi-Fi":
            return val

In [ ]:
get_current_wifi()

In [ ]:
def connect_wifi(nw):
    if get_current_wifi() == nw:
        print("Already on %s" % nw)
        return
    try:
        subprocess.check_call("netsh wlan connect name=\"%s\""%nw)
    except subprocess.CalledProcessError as e:
        raise NameError("No network named %r found" % nw) from None
    print("Switched to %s" % nw)

In [ ]:
connect_wifi("pbstech")

In [ ]:
connect_wifi("PBSBIOTECH")

In [1]:
from scripts import wlan

In [2]:
wlan.ensure_wifi('pbstech', 20)

Switched to pbstech


In [ ]:
import clipboard
clipboard.copy(repr(clipboard.paste()))

In [ ]:
import os
os.path.dirname("C:\\foo\\bar")